## Computing CAS scores

Let $M$ be the membership matrix of dimension $n \times k$ where $n$ is the number of nodes and $k$ the number of communities such that $M_{i,j}=1$ iff node $i$ is in community $j$, else 0.

Recall that each vertex can have multiple (or no) membership. A partition yields a single membership per node (row sums are all 1).

#### IEF and Beta

$IEF_A(v) = \frac{deg_A(v)}{deg(v)}$

$Beta_A(v) =  IEF_A(v) - p(A)$

where:

* $p(A) = vol(A)/vol(V)$
* $deg_A(v)$: number of neighbours $w$ of node $v$ with $w \in A$
* $deg(v)$: is the usual degree
* $vol(A)$: sum of degree of all nodes $w \in A$
* $vol(V)$: sum of degree of all nodes, i.e. twice the number of edges in the graph

Computing all $IEF_A(v)$ is a special case of the following with $\alpha=1$:

$D^{-1} \left((1-\alpha)A^2 + \alpha A \right) M$

where $D$ is the diagonal matrix of degrees, $A$ is the adjacency and $M$ if the community membership matrix.


#### C

$C_A(v) = \frac{Beta_A(v) * \sqrt{deg(v)}}{\sqrt{p(A)(1-p(A))}}$

This measures the number of standard deviations away from the expected number of edges from node $v$ to community $A$, assuming a binomial distribution.

#### P-values

This scores amounts to one minus the p-value for the number of edge into some community assuming a binomial distribution, that is:

$P_A(v) = Pr(bin(n=deg(v), p=p(A)) \le deg_A(v)$

#### Membership matrix M

In practice, we drop negative values to keep all matrices above sparse.

Given a score matrix S (one of the above), we can replace the membership matrix with a new matrix $M$ such that $M_{i,j} = 1$ iff $S_j(i) \ge s^*$ else 0.

Given this updated matrix $M$, we can re-compute the score matrix. 
This process can be repeated several times.


In [1]:
import pickle
import igraph as ig
from collections import Counter
from matplotlib import pyplot as plt
from abcd_graph import ABCDGraph, ABCDParams
from abcd_graph.utils import seed
from CAS import *

In [2]:
seed(123)
params = ABCDParams(
    vcount=1000, 
    gamma=2.5,
    min_degree=5,
    max_degree=30,
    beta=1.5,
    min_community_size=30,
    max_community_size=200,
    xi=0.3,
    num_outliers=50,
)
G = ABCDGraph(params)
G.build()
g = G.exporter.to_igraph()
## outliers have community == -1, map to 0-based
g.vs['ground_truth_community'] = np.array(g.vs['ground_truth_community']) + 1


In [3]:
### Get initial partition and adjacency
L = g.community_leiden(objective_function='modularity').membership
M = partition2sparse(L)
A = g.get_adjacency_sparse()

### Compute Beta and C scores; also get degrees w.r.t. initial partition
IEF, Beta, C, Pv, DegPart = CAS(A, M, alpha=1)

## Number of Outliers 
Counter(np.array(g.vs['ground_truth_community'])==0)

Counter({False: 950, True: 50})

In [4]:
### re-compute membership matrix given score and threshold
M = score_to_memberships(IEF, DegPart, threshold=.5, min_deg_in=2)
### number of memberships
print(Counter(M.sum(axis=1).flatten().tolist()[0]))
### compare to GT outliers
outliers = np.where(np.array(M.sum(axis=1).flatten().tolist()[0])==0)[0]
print('outliers found:',len(outliers))
print('ground-truth outliers found:',sum([g.vs[i]['ground_truth_community']==0 for i in outliers]))


Counter({1: 950, 0: 50})
outliers found: 50
ground-truth outliers found: 46


In [5]:
### re-compute membership matrix given score and threshold
M = score_to_memberships(Beta, DegPart, threshold=.4, min_deg_in=2)
### number of memberships
print(Counter(M.sum(axis=1).flatten().tolist()[0]))
### compare to GT outliers
outliers = np.where(np.array(M.sum(axis=1).flatten().tolist()[0])==0)[0]
print('outliers found:',len(outliers))
print('ground-truth outliers found:',sum([g.vs[i]['ground_truth_community']==0 for i in outliers]))


Counter({1: 949, 0: 51})
outliers found: 51
ground-truth outliers found: 46


In [6]:
### re-compute membership matrix given score and threshold
M = score_to_memberships(C, DegPart, threshold=2.5, min_deg_in=2)
### number of memberships
print(Counter(M.sum(axis=1).flatten().tolist()[0]))
### compare to GT outliers
outliers = np.where(np.array(M.sum(axis=1).flatten().tolist()[0])==0)[0]
print('outliers found:',len(outliers))
print('ground-truth outliers found:',sum([g.vs[i]['ground_truth_community']==0 for i in outliers]))


Counter({1: 934, 0: 46, 2: 20})
outliers found: 46
ground-truth outliers found: 31


In [7]:
### re-compute membership matrix given score and threshold
M = score_to_memberships(Pv, DegPart, threshold=.95, min_deg_in=2)
### number of memberships
print(Counter(M.sum(axis=1).flatten().tolist()[0]))
### compare to GT outliers
outliers = np.where(np.array(M.sum(axis=1).flatten().tolist()[0])==0)[0]
print('outliers found:',len(outliers))
print('ground-truth outliers found:',sum([g.vs[i]['ground_truth_community']==0 for i in outliers]))


Counter({1: 938, 0: 46, 2: 16})
outliers found: 46
ground-truth outliers found: 30


In [8]:
### loop - update C matrix
M = score_to_memberships(C, DegPart, threshold=2.5, min_deg_in=2)
print(Counter(M.sum(axis=1).flatten().tolist()[0]))
_I, _Beta, C, _S, _Deg = CAS(A, M, alpha=1)
M = score_to_memberships(C, DegPart, threshold=2.5, min_deg_in=2)
print(Counter(M.sum(axis=1).flatten().tolist()[0]))
_I, _Beta, C, _S, _Deg = CAS(A, M, alpha=1)
M = score_to_memberships(C, DegPart, threshold=2.5, min_deg_in=2)
print(Counter(M.sum(axis=1).flatten().tolist()[0]))
### compare to GT outliers
outliers = np.where(np.array(M.sum(axis=1).flatten().tolist()[0])==0)[0]
print('outliers found:',len(outliers))
print('ground-truth outliers found:',sum([g.vs[i]['ground_truth_community']==0 for i in outliers]))


Counter({1: 934, 0: 46, 2: 20})
Counter({1: 930, 0: 55, 2: 15})
Counter({1: 932, 0: 48, 2: 20})
outliers found: 48
ground-truth outliers found: 33
